# Week 2 - Day 1 & 2 - Challenge

This is an implementation of a three-way IELTS Band 9 conversation between LLMs: `gpt-4o-mini`, `llama3.2`, and `open-mistral-nemo`.

## 0. Import Libraries

In [ ]:
import os
import time

from dotenv import load_dotenv
import gradio as gr
from mistralai import Mistral
from openai import OpenAI
from typing import Generator

## 1. Load Environment Variables and Constants

In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
mistral_api_key = os.getenv("MISTRAL_API_KEY")

gpt_client = OpenAI(api_key=openai_api_key)
llama_client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama"
    )
mistral_client = Mistral(api_key=mistral_api_key)

## 2. Create System Prompts

In [ ]:
system_prompt_1 = """You are an articulate intellectual with IELTS Band 9 \
speaking proficiency. Use sophisticated vocabulary, complex grammar, and \
impeccable fluency. Demonstrate precise reasoning with nuanced expressions \
and seamless transitions. Maintain your sharp, decisive personality while \
elaborating on topics with well-structured arguments and strategic insights."""

system_prompt_2 = """You are an empathetic mediator with IELTS Band 9 \
speaking proficiency. Employ diverse vocabulary and complex structures with \
consistent accuracy. Your responses should flow naturally with advanced \
connectives while demonstrating emotional intelligence. Balance analytical \
depth with interpersonal warmth, adapting your register appropriately while \
maintaining your diplomatic, bridge-building nature."""

system_prompt_3 = """You are a perceptive observer with IELTS Band 9 \
speaking proficiency and subtle wit. Use precise vocabulary and idiomatic \
expressions while maintaining your sardonic perspective. Offer nuanced \
insights with sophisticated language and natural transitions. Balance \
academic precision with conversational authenticity, preserving your \
skeptical yet insightful personality when delivering unexpected \
observations."""

system_prompts = {
    "decisive": system_prompt_1,
    "empathic": system_prompt_2,
    "sardonic": system_prompt_3
    }

## 3. Create LLM Functions

### 3.1. `gpt-4o-mini`

In [ ]:
def call_gpt(
    client: OpenAI,
    system_prompt: str,
    topic: str,
    gpt_history: list[str],
    llama_history: list[str],
    mistral_history: list[str]
    ) -> str:
    system_prompt = f"""{system_prompt}
    
    IMPORTANT INSTRUCTIONS:
    1. You are GPT. You are in a three-way IELTS Band 9 conversation with LLAMA and MISTRAL.
    2. DO NOT prefix your response with "GPT:" - just respond directly.
    3. DO NOT roleplay as LLAMA or MISTRAL.
    4. DO NOT refer to yourself in the third person.
    6. The topic of the conversation is {topic}.
    7. Keep your response focused on the topic of {topic} and in line with IELTS Band 9 speaking critera.
    8. Keep your response short, relevant, and concise."""
    
    messages = [{"role": "system", "content": system_prompt}]
    
    max_length = max(len(gpt_history), len(llama_history), len(mistral_history))
    
    # Handle initial messages before the conversation starts
    if len(gpt_history) > 0:
        messages.append({"role": "assistant", "content": gpt_history[0]})
    
    # Handle messages from other participants in proper order
    for i in range(1, max_length):
        if i < len(llama_history):
            messages.append({"role": "user", "content": llama_history[i-1]})
        if i < len(mistral_history):
            messages.append({"role": "user", "content": mistral_history[i-1]})
        if i < len(gpt_history):
            messages.append({"role": "assistant", "content": gpt_history[i]})
    
    # Handle the most recent messages if they exist
    if len(llama_history) >= max_length:
        messages.append({"role": "user", "content": llama_history[-1]})
    if len(mistral_history) >= max_length:
        messages.append({"role": "user", "content": mistral_history[-1]})
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        stream=False,
        temperature=0.5
        )
    return response.choices[0].message.content

### 3.2. `llama3.2`

In [ ]:
def call_llama(
    client: OpenAI,
    system_prompt: str,
    topic: str,
    gpt_history: list[str],
    llama_history: list[str],
    mistral_history: list[str]
    ) -> str:
    system_prompt = f"""{system_prompt}
    
    IMPORTANT INSTRUCTIONS:
    1. You are LLAMA. You are in a three-way IELTS Band 9 conversation with GPT and MISTRAL.
    2. DO NOT prefix your response with "LLAMA:" - just respond directly.
    3. DO NOT roleplay as GPT or MISTRAL.
    4. DO NOT refer to yourself in the third person.
    5. Only respond as yourself.
    6. The topic of the conversation is {topic}.
    7. Keep your response focused on the topic of {topic} and in line with IELTS Band 9 speaking critera.
    8. Keep your response short, relevant, and concise."""
    
    messages = [{"role": "system", "content": system_prompt}]
    
    max_length = max(len(gpt_history), len(llama_history), len(mistral_history))
    
    # Handle initial messages before the conversation starts
    if len(llama_history) > 0:
        messages.append({"role": "assistant", "content": llama_history[0]})
    
    # Handle messages from other participants in proper order
    for i in range(1, max_length):
        if i < len(gpt_history):
            messages.append({"role": "user", "content": gpt_history[i-1]})
        if i < len(mistral_history):
            messages.append({"role": "user", "content": mistral_history[i-1]})
        if i < len(llama_history):
            messages.append({"role": "assistant", "content": llama_history[i]})
    
    # Handle the most recent messages if they exist
    if len(gpt_history) >= max_length:
        messages.append({"role": "user", "content": gpt_history[-1]})
    if len(mistral_history) >= max_length:
        messages.append({"role": "user", "content": mistral_history[-1]})
    
    response = client.chat.completions.create(
        model="llama3.2",
        messages=messages,
        stream=False,
        temperature=0.5
        )
    return response.choices[0].message.content

### 3.3. `open-mistral-nemo`

In [ ]:
def call_mistral(
    client: Mistral,
    system_prompt: str,
    topic: str,
    gpt_history: list[str],
    llama_history: list[str],
    mistral_history: list[str]
    ) -> str:
    system_prompt = f"""{system_prompt}
    
    IMPORTANT INSTRUCTIONS:
    1. You are MISTRAL. You are in a three-way IELTS Band 9 conversation with GPT and LLAMA.
    2. DO NOT prefix your response with "MISTRAL:" - just respond directly.
    3. DO NOT roleplay as GPT or LLAMA.
    4. DO NOT refer to yourself in the third person.
    5. Only respond as yourself.
    6. The topic of the conversation is {topic}.
    7. Keep your response focused on the topic of {topic} and in line with IELTS Band 9 speaking critera.
    8. Keep your response short, relevant, and concise."""
    
    messages = [{"role": "system", "content": system_prompt}]
    
    max_length = max(len(gpt_history), len(llama_history), len(mistral_history))
    
    # Handle initial messages before the conversation starts
    if len(mistral_history) > 0:
        messages.append({"role": "assistant", "content": mistral_history[0]})
    
    # Handle messages from other participants in proper order
    for i in range(1, max_length):
        if i < len(gpt_history):
            messages.append({"role": "user", "content": gpt_history[i-1]})
        if i < len(llama_history):
            messages.append({"role": "user", "content": llama_history[i-1]})
        if i < len(mistral_history):
            messages.append({"role": "assistant", "content": mistral_history[i]})
    
    # Handle the most recent messages if they exist
    if len(gpt_history) >= max_length:
        messages.append({"role": "user", "content": gpt_history[-1]})
    if len(llama_history) >= max_length:
        messages.append({"role": "user", "content": llama_history[-1]})
    
    response = client.chat.complete(
        model="open-mistral-nemo",
        messages=messages
        )
    return response.choices[0].message.content

## 4. Begin Conversation

### 4.1. Display by `print()`

In [ ]:
def begin_conversation(
    topic: str,
    conversation_rounds: int,
    gpt_character: str,
    llama_character: str,
    mistral_character: str
    ) -> None:
    try:
        conversation_rounds = int(conversation_rounds)
    except ValueError:
        return "Error: conversation_rounds must be a number!"
    
    gpt_system_prompt = system_prompts[gpt_character]
    llama_system_prompt = system_prompts[llama_character]
    mistral_system_prompt = system_prompts[mistral_character]
    
    gpt_history = ["Hello!"]
    llama_history = ["Good Afternoon!"]
    mistral_history = ["Greetings!"]

    print(f"**GPT:** {gpt_history[0]}\n\n")
    print(f"**LLAMA:** {llama_history[0]}\n\n")
    print(f"**MISTRAL:** {mistral_history[0]}\n\n")

    for _ in range(1, conversation_rounds):
        gpt_response = call_gpt(
            client=gpt_client,
            topic=topic,
            system_prompt=gpt_system_prompt,
            gpt_history=gpt_history,
            llama_history=llama_history,
            mistral_history=mistral_history
            )
        gpt_history.append(gpt_response)
        
        llama_response = call_llama(
            client=llama_client,
            topic=topic,
            system_prompt=llama_system_prompt,
            gpt_history=gpt_history,
            llama_history=llama_history,
            mistral_history=mistral_history
            )
        llama_history.append(llama_response)
        
        mistral_response = call_mistral(
            client=mistral_client,
            topic=topic,
            system_prompt=mistral_system_prompt,
            gpt_history=gpt_history,
            llama_history=llama_history,
            mistral_history=mistral_history
            )
        mistral_history.append(mistral_response)
        
        print(f"**GPT:** {gpt_response}\n\n")
        print(f"**LLAMA:** {llama_response}\n\n")
        print(f"**MISTRAL:** {mistral_response}\n\n")

begin_conversation(
    topic="money",
    conversation_rounds=3,
    gpt_character="decisive",
    llama_character="empathic",
    mistral_character="sardonic"
    )

### 4.2. Display by `gradio.Interface()`

In [ ]:
def begin_conversation(
    topic: str,
    conversation_rounds: int,
    gpt_character: str,
    llama_character: str,
    mistral_character: str
    ) -> Generator[str, None, None]:
    try:
        conversation_rounds = int(conversation_rounds)
    except ValueError:
        return "Error: conversation_rounds must be a number!"
    
    gpt_system_prompt = system_prompts[gpt_character]
    llama_system_prompt = system_prompts[llama_character]
    mistral_system_prompt = system_prompts[mistral_character]
    
    stream = ""
    gpt_history = ["Hello!"]
    llama_history = ["Good Afternoon!"]
    mistral_history = ["Greetings!"]
    
    stream += f"\n\n**GPT:** "
    yield stream
    for char in gpt_history[0]:
        stream += char
        yield stream
        time.sleep(0.02)
    
    stream += f"\n\n**LLAMA:** "
    yield stream
    for char in llama_history[0]:
        stream += char
        yield stream
        time.sleep(0.02)
    
    stream += f"\n\n**MISTRAL:** "
    yield stream
    for char in mistral_history[0]:
        stream += char
        yield stream
        time.sleep(0.02)

    for _ in range(1, conversation_rounds):
        gpt_response = call_gpt(
            client=gpt_client,
            topic=topic,
            system_prompt=gpt_system_prompt,
            gpt_history=gpt_history,
            llama_history=llama_history,
            mistral_history=mistral_history
            )
        gpt_history.append(gpt_response)
        stream += f"\n\n**GPT:** "
        yield stream
        for char in gpt_response:
            stream += char
            yield stream
            time.sleep(0.02)
        
        llama_response = call_llama(
            client=llama_client,
            topic=topic,
            system_prompt=llama_system_prompt,
            gpt_history=gpt_history,
            llama_history=llama_history,
            mistral_history=mistral_history
            )
        llama_history.append(llama_response)
        stream += f"\n\n**LLAMA:** "
        yield stream
        for char in llama_response:
            stream += char
            yield stream
            time.sleep(0.02)
        
        mistral_response = call_mistral(
            client=mistral_client,
            topic=topic,
            system_prompt=mistral_system_prompt,
            gpt_history=gpt_history,
            llama_history=llama_history,
            mistral_history=mistral_history
            )
        mistral_history.append(mistral_response)
        stream += f"\n\n**MISTRAL:** "
        yield stream
        for char in mistral_response:
            stream += char
            yield stream
            time.sleep(0.02)
        
gr.Interface(
    fn=begin_conversation,
    inputs=[
        gr.Textbox(label="Topic"),
        gr.Textbox(label="Conversation Rounds"),
        gr.Dropdown(["decisive", "empathic", "sardonic"], label="GPT Character Type"),
        gr.Dropdown(["decisive", "empathic", "sardonic"], label="LLAMA Character Type"),
        gr.Dropdown(["decisive", "empathic", "sardonic"], label="MISTRAL Character Type"),
        ],
    outputs=[gr.Markdown(label="Response")],
    flagging_mode="never"
    ).launch()